# Sample Experiments

In this notebook, we will explore simple ways for conducting experiments with the different instances. First we will download some debris datasets. Note that you can choose any data from https://celestrak.com/NORAD/elements/ or the full TLE set https://www.space-track.org/

In [1]:
import urllib.request
import os

def download(url, file_name):
    if not os.path.exists(file_name):
        print("downloading data from " + str(url))
        with open(file_name, "wb") as file:
            response = urllib.request.urlopen(url)
            file.write(response.read())
    
PATH_TO_SATCAT = "data/satcat.txt"
PATH_TO_IRIDIUM_TLE = "data/iridium-33-debris.txt"
PATH_TO_COSMOS_TLE = "data/cosmos-2251-debris.txt"
PATH_TO_FEN_TLE = "data/fen-debris.txt"
ALL_PATHS_TLE = [PATH_TO_IRIDIUM_TLE, PATH_TO_IRIDIUM_TLE, PATH_TO_FEN_TLE]

if not os.path.exists("data"):
    print("Creating data folder")
    os.makedirs("data")
    
download("https://celestrak.com/pub/satcat.txt", PATH_TO_SATCAT)
download("https://celestrak.com/NORAD/elements/iridium-33-debris.txt", PATH_TO_IRIDIUM_TLE)
download("https://celestrak.com/NORAD/elements/cosmos-2251-debris.txt", PATH_TO_COSMOS_TLE)
download("https://celestrak.com/NORAD/elements/1999-025.txt", PATH_TO_FEN_TLE)

# Experiment 1: Find optimal solutions for static TSP

Assuming the traveling costs do not change over time, we can compute solutions for the three datasets with a fixed number of debris objects in the instances

In [2]:
from preprocessing import data_processing
from optimization import mips_static

In [3]:
MAX_DEBRIS = 50
BUDGET = 1000

for path in ALL_PATHS_TLE:
    print("Conducting experiment for " + path[5:][:-4])
    A, P = data_processing.compute_static_TSP_instance(PATH_TO_SATCAT, path, MAX_DEBRIS)
    res = mips_static.TSP_static(A, P, BUDGET)
    print("\n\n ---- Resulting total radar cross section: " + str(res.objVal) + " m^2 ----\n\n")

Conducting experiment for iridium-33-debris
Preprocessing indices for three impulse approximation



Computing cost approximations for static TSP



Academic license - for non-commercial use only
Optimize a model with 2704 rows, 5151 columns and 17903 nonzeros
Variable types: 2550 continuous, 2601 integer (2601 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [5e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 2185 rows and 4317 columns
Presolve time: 0.03s
Presolved: 519 rows, 834 columns, 2956 nonzeros
Variable types: 407 continuous, 427 integer (427 binary)
Found heuristic solution: objective 0.0480000

Root relaxation: objective 6.792884e+00, 650 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.79288    0   43    0.04800    6.79288      -     -    0s
H    0     0                       0.1180000    6.79288  5657%     -    0s
     0     0    6.73821    0   54    0.11800    6.73821  5610%     -    0s
   


Computing cost approximations for static TSP



Optimize a model with 2704 rows, 5151 columns and 17903 nonzeros
Variable types: 2550 continuous, 2601 integer (2601 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [5e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 2185 rows and 4317 columns
Presolve time: 0.03s
Presolved: 519 rows, 834 columns, 2956 nonzeros
Variable types: 407 continuous, 427 integer (427 binary)
Found heuristic solution: objective 0.0480000

Root relaxation: objective 6.792884e+00, 650 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.79288    0   43    0.04800    6.79288      -     -    0s
H    0     0                       0.1180000    6.79288  5657%     -    0s
     0     0    6.73821    0   54    0.11800    6.73821  5610%     -    0s
     0     0    6.73821    0   54    0.11800    6.


Computing cost approximations for static TSP



Optimize a model with 2704 rows, 5151 columns and 17903 nonzeros
Variable types: 2550 continuous, 2601 integer (2601 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [9e-02, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 0.1820000
Presolve removed 2466 rows and 4818 columns
Presolve time: 0.02s
Presolved: 238 rows, 333 columns, 1332 nonzeros
Variable types: 170 continuous, 163 integer (163 binary)

Root relaxation: objective 1.895448e+00, 116 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    1.89545    0   10    0.18200    1.89545   941%     -    0s
H    0     0                       0.2435000    1.89545   678%     -    0s
H    0     0                       0.7230000    1.89502   162%     -    0s
     0     0    1.88955    0   10    0.72300    1.

# Experiment 2: Varying the static TSP formulation

In order to test the performance of the different TSP formulations, we can try them on the same data set.

In [4]:
MAX_DEBRIS = 30
BUDGET = 1000
A, P = data_processing.compute_static_TSP_instance(PATH_TO_SATCAT, PATH_TO_IRIDIUM_TLE, MAX_DEBRIS)

for sec in ["F1","F3", "seq", "T1", "T2"]:
    print("\n\nConducting experiment for sec type " + sec + "\n\n")
    res = mips_static.TSP_static(A, P, BUDGET, SEC_TYPE = sec)

Preprocessing indices for three impulse approximation



Computing cost approximations for static TSP





Conducting experiment for sec type F1


Optimize a model with 1024 rows, 1891 columns and 6543 nonzeros
Variable types: 930 continuous, 961 integer (961 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [7e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 0.1455000
Presolve removed 806 rows and 1555 columns
Presolve time: 0.01s
Presolved: 218 rows, 336 columns, 1179 nonzeros
Variable types: 159 continuous, 177 integer (177 binary)

Root relaxation: objective 6.516203e+00, 260 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.51620    0   27    0.14550    6.51620  4378%     -    0s
     0     0    6.28340    0   26    0.14550    6.28340  4218%     -    0s
H    0     0                       0.6621000    6.28340   849%     -    0s
H    0     0 

     0     0    4.59998    0   45    3.97480    4.59998  15.7%     -    0s
     0     0    4.54291    0   34    3.97480    4.54291  14.3%     -    0s
     0     0    4.53862    0   43    3.97480    4.53862  14.2%     -    0s
     0     0    4.52050    0   41    3.97480    4.52050  13.7%     -    0s
     0     0    4.51087    0   40    3.97480    4.51087  13.5%     -    0s
     0     0    4.51087    0   40    3.97480    4.51087  13.5%     -    0s
     0     0    4.48500    0   42    3.97480    4.48500  12.8%     -    0s
     0     0    4.48447    0   44    3.97480    4.48447  12.8%     -    0s
     0     0    4.47259    0   54    3.97480    4.47259  12.5%     -    0s
     0     0    4.47257    0   48    3.97480    4.47257  12.5%     -    0s
     0     0    4.47242    0   48    3.97480    4.47242  12.5%     -    0s
     0     0    4.47235    0   54    3.97480    4.47235  12.5%     -    0s
     0     0    4.47235    0   49    3.97480    4.47235  12.5%     -    0s
     0     0    4.47235  

Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [7e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 846 rows and 24220 columns
Presolve time: 0.08s
Presolved: 270 rows, 5571 columns, 20288 nonzeros
Variable types: 5394 continuous, 177 integer (177 binary)

Root relaxation: objective 6.662087e+00, 816 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.66209    0   34   -0.00000    6.66209      -     -    0s
H    0     0                       0.5585000    6.66209  1093%     -    0s
     0     0    6.56933    0   49    0.55850    6.56933  1076%     -    0s
H    0     0                       0.6992000    6.56933   840%     -    0s
     0     0    6.55082    0   44    0.69920    6.55082   837%     -    0s
     0     0    6.5504

# Experiment 3: Covering Salesman problem for static TSP

Instead of approaching a debris object, we will compute a "closeness" list for objects that are close to each other. The idea is that the removal is not performed of the chaser rocket itself, but rather through a microsatellite that exits the chaser rocket (mothership).

In this experiment, we will vary the closeness parameter and see the impact on the results

In [5]:
MAX_DEBRIS = 30
BUDGET = 1000
A, P = data_processing.compute_static_TSP_instance(PATH_TO_SATCAT, PATH_TO_IRIDIUM_TLE, nMaxNodes = MAX_DEBRIS)

for R in [25, 50, 75]:
    print("\n\n-- Conducting experiment for R = " + str(R) + " --\n\n")
    C = data_processing.compute_closeness(A, R)
    mips_static.cov_TSP_with_covering_cost(A, P, C, BUDGET)
    print("\n\n ---- Resulting total radar cross section: " + str(res.objVal) + " m^2 ----\n\n")

Preprocessing indices for three impulse approximation



Computing cost approximations for static TSP





-- Conducting experiment for R = 25 --


Changed value of parameter TimeLimit to 10800.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 1098 rows, 2852 columns and 6714 nonzeros
Variable types: 930 continuous, 1922 integer (1922 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [7e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 0.6621000
Presolve removed 850 rows and 2503 columns
Presolve time: 0.01s
Presolved: 248 rows, 349 columns, 1266 nonzeros
Variable types: 159 continuous, 190 integer (190 binary)

Root relaxation: objective 6.577817e+00, 261 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.57782    0   57    0.66210    6.57782   893%     -    0s
     0     0    6.53320    0   45    0.66210    6.53320 

     0     0    5.14008    0   68    3.30400    5.14008  55.6%     -    0s
H    0     0                       3.8877000    5.14008  32.2%     -    0s
     0     0    5.13870    0   67    3.88770    5.13870  32.2%     -    0s
     0     0    5.13272    0   59    3.88770    5.13272  32.0%     -    0s
     0     0    5.13272    0   59    3.88770    5.13272  32.0%     -    0s
     0     0    5.12640    0   67    3.88770    5.12640  31.9%     -    0s
     0     0    5.12640    0   67    3.88770    5.12640  31.9%     -    0s
     0     2    5.12640    0   67    3.88770    5.12640  31.9%     -    0s
H  121    31                       3.9748000    4.61270  16.0%  16.5    0s

Cutting planes:
  Gomory: 3
  Cover: 1
  Implied bound: 15
  MIR: 21
  Flow cover: 29
  Zero half: 13
  Network: 7

Explored 344 nodes (5292 simplex iterations) in 0.37 seconds
Thread count was 8 (of 8 available processors)

Solution count 4: 3.9748 3.8877 3.304 0.6621 

Optimal solution found (tolerance 1.00e-04)
Best obj

# Experiment 4: Find optimal solutions for dynamic TSP

In [6]:
from optimization import mips_dynamic

In [7]:
MAX_DEBRIS = 30
BUDGET = 1000

# julian date from 2000
START_DAY = 7350

# number of possible transfer points
EPOCHS = 10

# number of days between transfers
STEP_SIZE = 7

# time needed for a transfer, here 1 week
TRANSFER_DURATION = 1


for path in ALL_PATHS_TLE:
    print("Conducting experiment for " + path[5:][:-4])
    A, P, par = data_processing.compute_dynamic_TSP_instance(SATCAT_PATH = "data/satcat.txt", TLE_PATH = path, startDay = START_DAY, nEpochs = EPOCHS, step_size = STEP_SIZE, nMaxNodes = MAX_DEBRIS)
    res = mips_dynamic.TSP_dynamic(A, P, transfer_duration = TRANSFER_DURATION, budget = BUDGET, SEC_TYPE = "DT1")
    print("\n\n ---- Resulting total radar cross section: " + str(res.objVal) + " m^2 ----\n\n")

Conducting experiment for iridium-33-debris
Preprocessing indices for dynamic three impulse approximation



Computing cost approximations for dynamic TSP



Optimize a model with 93 rows, 9331 columns and 43592 nonzeros
Variable types: 0 continuous, 9331 integer (9331 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [7e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 15 rows and 8285 columns
Presolve time: 0.03s
Presolved: 78 rows, 1046 columns, 4903 nonzeros
Found heuristic solution: objective 0.1079000
Variable types: 0 continuous, 1046 integer (1046 binary)

Root relaxation: objective 5.175019e+00, 139 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.17502    0   38    0.10790    5.17502  4696%     -    0s
H    0     0                       3.7977000    5.17502  36.3%     -    0s
H    0     0                       3.8877000    5.17499  33.1%     -    0s
     0    


Computing cost approximations for dynamic TSP



Optimize a model with 93 rows, 9331 columns and 43592 nonzeros
Variable types: 0 continuous, 9331 integer (9331 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [7e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 15 rows and 8285 columns
Presolve time: 0.04s
Presolved: 78 rows, 1046 columns, 4903 nonzeros
Found heuristic solution: objective 0.1079000
Variable types: 0 continuous, 1046 integer (1046 binary)

Root relaxation: objective 5.175019e+00, 139 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.17502    0   38    0.10790    5.17502  4696%     -    0s
H    0     0                       3.7977000    5.17502  36.3%     -    0s
H    0     0                       3.8877000    5.17499  33.1%     -    0s
     0    


Computing cost approximations for dynamic TSP



Optimize a model with 93 rows, 9331 columns and 43592 nonzeros
Variable types: 0 continuous, 9331 integer (9331 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e-01, 6e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 30 rows and 7587 columns
Presolve time: 0.05s
Presolved: 63 rows, 1744 columns, 8326 nonzeros
Found heuristic solution: objective 0.1480000
Variable types: 0 continuous, 1744 integer (1744 binary)

Root relaxation: objective 2.353670e+00, 326 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.35367    0   29    0.14800    2.35367  1490%     -    0s
H    0     0                       1.2080000    2.35367  94.8%     -    0s
H    0     0                       1.3975000    2.35367  68.4%     -    0s
H    0    

# Experiment 5: Vary time parameters for dynamic TSP

All parameters can be varied and yield different results based on the mission design choice.

In [8]:
MAX_DEBRIS = 30
BUDGET = 1000

# julian date from 2000
START_DAY = 7350

# number of days between transfers
STEP_SIZE = 7

for nEpochs in [10,15,20]:
    for transfer_dur in [1,2]:
    
        print("Conducting experiment for " + path[5:][:-4])
        A, P, par = data_processing.compute_dynamic_TSP_instance(SATCAT_PATH = "data/satcat.txt", TLE_PATH = PATH_TO_IRIDIUM_TLE, startDay = START_DAY, nEpochs = nEpochs, step_size = STEP_SIZE, nMaxNodes = MAX_DEBRIS)
        res = mips_dynamic.TSP_dynamic(A, P, transfer_duration = transfer_dur, budget = BUDGET, SEC_TYPE = "DT1")
        print("\n\n ---- Resulting total radar cross section: " + str(res.objVal) + " m^2 ----\n\n")

Conducting experiment for fen-debris
Preprocessing indices for dynamic three impulse approximation



Computing cost approximations for dynamic TSP



Optimize a model with 93 rows, 9331 columns and 43592 nonzeros
Variable types: 0 continuous, 9331 integer (9331 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [7e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 15 rows and 8285 columns
Presolve time: 0.04s
Presolved: 78 rows, 1046 columns, 4903 nonzeros
Found heuristic solution: objective 0.1079000
Variable types: 0 continuous, 1046 integer (1046 binary)

Root relaxation: objective 5.175019e+00, 139 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.17502    0   38    0.10790    5.17502  4696%     -    0s
H    0     0                       3.7977000    5.17502  36.3%     -    0s
H    0     0                       3.8877000    5.17499  33.1%     -    0s
     0    


Computing cost approximations for dynamic TSP



Optimize a model with 93 rows, 9331 columns and 43592 nonzeros
Variable types: 0 continuous, 9331 integer (9331 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [7e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 15 rows and 8598 columns
Presolve time: 0.06s
Presolved: 78 rows, 733 columns, 3287 nonzeros
Found heuristic solution: objective 0.1079000
Variable types: 0 continuous, 733 integer (733 binary)

Root relaxation: objective 4.400690e+00, 203 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.40069    0   21    0.10790    4.40069  3978%     -    0s
H    0     0                       3.7977000    4.40069  15.9%     -    0s
     0     0    4.18151    0   25    3.79770    4.18151  10.1%     -    0s
H    0     0 

...


Optimize a model with 93 rows, 13981 columns and 66242 nonzeros
Variable types: 0 continuous, 13981 integer (13981 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [7e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 15 rows and 12593 columns
Presolve time: 0.05s
Presolved: 78 rows, 1388 columns, 6591 nonzeros
Found heuristic solution: objective 0.1079000
Variable types: 0 continuous, 1388 integer (1388 binary)

Root relaxation: objective 4.965208e+00, 200 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.96521    0   30    0.10790    4.96521  4502%     -    0s
H    0     0                       0.9165000    4.96521   442%     -    0s
H    0     0                       3.8877000    4.96521  27.7%     -    0s
     0


Computing cost approximations for dynamic TSP



Optimize a model with 93 rows, 18631 columns and 88892 nonzeros
Variable types: 0 continuous, 18631 integer (18631 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [7e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 15 rows and 16439 columns
Presolve time: 0.07s
Presolved: 78 rows, 2192 columns, 10633 nonzeros
Found heuristic solution: objective 0.1079000
Variable types: 0 continuous, 2192 integer (2192 binary)

Root relaxation: objective 5.535480e+00, 158 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.53548    0   41    0.10790    5.53548  5030%     -    0s
H    0     0                       3.8877000    5.53548  42.4%     -    0s
     0     0    5.52354    0   43    3.88770    5.52354  42.1%     -    0s
     


Computing cost approximations for dynamic TSP



Optimize a model with 93 rows, 18631 columns and 88892 nonzeros
Variable types: 0 continuous, 18631 integer (18631 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [7e-02, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+03, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 15 rows and 16675 columns
Presolve time: 0.08s
Presolved: 78 rows, 1956 columns, 9451 nonzeros
Found heuristic solution: objective 0.1079000
Variable types: 0 continuous, 1956 integer (1956 binary)

Root relaxation: objective 5.286694e+00, 189 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.28669    0   38    0.10790    5.28669  4800%     -    0s
H    0     0                       1.0553000    5.28669   401%     -    0s
H    0     0                       1.2787000    5.28669   313%     -    0s
H    0